In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools
import seaborn as sns
import numpy as np
import tensorflow as tf
import pandas as pd
import os

In [70]:
def print_logits (y,batch_size,nb_label):
    # Print a the label predicted
    #Input : 
    #y : Numpy array [batch_size][10] The data predicted
    #batch_size : Int The size of the batch
    #nb_label : Int The number of labels
    for i in range (0,batch_size):
        print("------------",i,"------------")
        for j in range (0,nb_label):
            if (y[i][j] != 0):
                print("prédiction : ",j,"pourcentage de certitude :","%.1f" % (y[i][j]*100) , " %")
                
def create_model():
    model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
  ])

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=[
                      keras.metrics.TruePositives(name='tp'),
                      keras.metrics.FalsePositives(name='fp'),
                      keras.metrics.TrueNegatives(name='tn'),
                      keras.metrics.FalseNegatives(name='fn'), 
                      keras.metrics.BinaryAccuracy(name='accuracy'),
                      keras.metrics.Precision(name='precision'),
                      keras.metrics.Recall(name='recall'),
                      keras.metrics.AUC(name='auc')])

    return model


In [17]:
team_data = pd.read_csv("teamfeatures.csv")
team_data = team_data.drop(columns = team_data.columns[0])

In [68]:
teams_to_predict = ["Lyon", "Paris SG"]

In [44]:
def names_to_features(names):
    away = team_data.loc[team_data[team_data.TeamName == names[0]].index, team_data.columns[:(len(team_data.columns)-1)]].values
    home = team_data.loc[team_data[team_data.TeamName == names[1]].index, team_data.columns[:(len(team_data.columns)-1)]].values
    feature_vec = np.array([away, home])
    vec_out = feature_vec.reshape((1,np.prod(feature_vec.shape)))
    return vec_out

In [59]:
team_data

,0,1,2,3,4,5,6,TeamName
0,0.631965,1.714772,0.291257,-0.089277,-0.474582,-0.489832,1.180820,Nantes
1,-0.962820,0.826610,0.074035,-0.514721,2.794413,-0.489832,-0.576492,Lille
2,-0.165427,0.604570,0.942921,-1.223794,-0.111361,-0.489832,-0.576492,Reims
3,1.429358,0.382529,0.074035,-1.081979,-0.111361,-0.489832,1.180820,Bastia
4,-0.564124,-1.060734,-1.012073,0.336166,0.978304,-0.489832,-0.576492,Evian Thonon Gaillard
5,2.625446,-0.505633,0.942921,-2.074681,-1.201026,-0.489832,-0.576492,Guingamp
6,0.233269,-0.616653,0.291257,-0.231092,0.251861,0.956749,-0.576492,Lyon
7,1.429358,1.492732,2.463472,-0.798350,0.615083,-0.007638,-0.576492,Montpellier
8,1.429358,1.159671,1.811808,0.903425,0.615083,-0.007638,-0.576492,Nice
9,-0.165427,-0.061552,0.074035,0.336166,-0.837804,1.438943,1.180820,St Etienne


In [60]:
checkpoint_path = "models/model1.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [61]:
model = create_model()
model.load_weights(checkpoint_path)

In [69]:
x_pred = names_to_features(teams_to_predict)

print_logits(model.predict(x = x_pred, batch_size = 1),1,3)

------------ 0 ------------
prédiction :  0 pourcentage de certitude : 31.5  %
prédiction :  1 pourcentage de certitude : 29.2  %
prédiction :  2 pourcentage de certitude : 39.2  %


In [46]:
x_pred = names_to_features(teams_to_predict)


In [48]:
x_pred.shape

(1, 14)

Envoie des réponses à la base